<a href="https://colab.research.google.com/github/supriyag123/PHD_Pub/blob/main/AGENTIC-MODULE5-RunCoordinatorDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# run_coordinator_demo.py
import numpy as np
import os

# Import your agents
from agents.robust_master_agent import create_robust_system
from agents.adaptive_window_agent import AdaptiveWindowAgent
from agents.coordinator_agent import CoordinatorAgent

if __name__ == "__main__":
    # =======================
    # 1. Setup models & agents
    # =======================
    models_dir = "/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/sensor/model"
    mlp_path = "/content/drive/MyDrive/PHD/2025/DGRNet-MLP-Versions/METROPM_MLP_model_Daily.keras"

    # Create sensor agents + master
    sensor_agents, master = create_robust_system(num_sensors=12, models_dir=models_dir)

    # Create window agent
    window_agent = AdaptiveWindowAgent(model_path=mlp_path)

    # Create coordinator
    coordinator = CoordinatorAgent(anomaly_weight=0.6, window_weight=0.4)

    # =======================
    # 2. Load hold-out dataset
    # =======================
    data_path = "/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/multivariate_long_sequences-TRAIN-Daily-DIRECT-VAR.npy"
    subsequences = np.load(data_path)  # shape: [n_samples, window_length, num_sensors]
    holdout = subsequences[-20:]       # last 20 sequences as test
    print(f"Loaded holdout data: {holdout.shape}")

    # =======================
    # 3. Run streaming demo
    # =======================
    for i, seq in enumerate(holdout, start=1):
        features = seq.flatten()

        # A. Get sensor master output
        master_out = master.process_system_input(seq)

        # B. Get window agent output
        window_out = window_agent.predict_window_size(features, seq)

        # C. Fuse in coordinator
        final_decision = coordinator.fuse(master_out, window_out)

        # =======================
        # 4. Print results
        # =======================
        print("\n" + "="*70)
        print(f"Sample {i}")
        print("- Master decision:", master_out["system_decisions"])
        print("- Window decision:", {
            "predicted_window": window_out.get("predicted_window"),
            "event": window_out.get("event_type")
        })
        print("- Final fused decision:", final_decision["alert_level"])
        print("- Scores:", final_decision["scores"])
        print("- Window agent flags:", final_decision["window_agent"])


✅ Loaded subsequences: (3627, 50, 12)
🚀 Creating robust system with 12 sensors
✅ AE model loaded for sensor 0
✅ AE model loaded for sensor 1
✅ AE model loaded for sensor 2
✅ AE model loaded for sensor 3
✅ AE model loaded for sensor 4
✅ AE model loaded for sensor 5
✅ AE model loaded for sensor 6
✅ AE model loaded for sensor 7
✅ AE model loaded for sensor 8
✅ AE model loaded for sensor 9
✅ AE model loaded for sensor 10
✅ AE model loaded for sensor 11
✅ Created system: 12/12 models loaded
▶️ Running streaming test on 100 hold-out subsequences

Step 1/100
System decision: {'system_anomaly': False, 'system_drift': False, 'system_needs_retrain': False, 'anomaly_rate': 0.0, 'drift_rate': 0.0, 'retrain_rate': 0.0}
  Sensor 0: anomaly=False drift=False score=0.0013 conf=0.67
  Sensor 1: anomaly=False drift=False score=0.0002 conf=0.67
  Sensor 2: anomaly=False drift=False score=0.0014 conf=0.67
  Sensor 3: anomaly=False drift=False score=0.0011 conf=0.67
  Sensor 4: anomaly=False drift=False sc

In [ ]:
from google.colab import drive
drive.mount('/content/drive')